In [1]:
from src.data_local import MergedDataPreprocessing
from cashe import get_cashed_input
import pandas as pd

## History tracking
Data hist: loaded on 23/05/2024 / loaded on 12/08/2024 / loaded on 09/09/2024

In [2]:
dir_date_day = '31-08'; last_date_str = '2024-07-10'

In [3]:
df_original = get_cashed_input(dir_date_day= dir_date_day,last_date_str= last_date_str)

In [4]:
len(df_original)

320054

In [5]:
df_original['OUTCOME'].value_counts()

OUTCOME
APPROVED    160428
REJECTED    137130
PARTIAL      22496
Name: count, dtype: int64

In [6]:
d1 = df_original['CREATION_DATE'].sort_values().iloc[0]
d2 = df_original['CREATION_DATE'].sort_values().iloc[-1]
print('Date ranges from:',str(d1)[:10],',to',str(d2)[:10])

Date ranges from: 2024-01-01 ,to 2024-07-08


In [7]:
df_original.ICD10.isna().sum()

50205

In [9]:
df_ref = pd.read_parquet('data/utilities/reference_format/reference.parquet')

cols = df_ref.columns
for i in range(len(cols)):
    col = cols[i]
    type_ = df_ref[col].dtype
    if col in df_original.columns:
        df_original[col] = df_original[col].astype(type_)

In [10]:
for col in df_original.columns:
    print(col,":",df_original[col].dtype)

REQUEST_DATE : datetime64[ns]
VISIT_NO : object
VISIT_DATE : datetime64[ns]
EMERGENCY_INDICATOR : object
PROVIDER_DEPARTMENT_CODE : int64
PROVIDER_DEPARTMENT : object
DOCTOR_SPECIALTY_CODE : float64
DOCTOR_CODE : int64
PATIENT_NO : int64
EPISODE_NO : int64
PATIENT_ID : int64
MEMBERSHIP_NO : object
PATIENT_ID_TYPE : float64
PATIENT_NATIONAL_IQAMA_ID : object
PATIENT_DOB : datetime64[ns]
PATIENT_AGE : int64
UNIT_OF_AGE : int64
PATIENT_NATIONALITY : object
PATIENT_MARITAL_STATUS : object
PATIENT_GENDER : object
PATIENT_MOBILE_NO : int64
PATIENT_BADGE_NO : object
HIS_INSURANCE_CODE : int64
POLICY_HOLDER_NO : object
CLAIM_TYPE : object
BATCH_NO : int64
TOTAL_CLAIMED_AMOUNT : float64
TOTAL_CLAIMED_AMOUNT_SAR : float64
TOTAL_NET_AMOUNT : float64
TOTAL_DISCOUNT : float64
TOTAL_DEDUCTIBLE : float64
TOTAL_PATIENT_VATAMOUNT : float64
TOTAL_NET_VAT_AMOUNT : float64
CLAIM_NO : object
STAT_END_DATE : datetime64[ns]
AMEND_BY_USER : int64
AMEND_LAST_DATE : datetime64[ns]
PAYER_LICENSE : int64
DOCTOR_L

In [11]:
preprocessing = MergedDataPreprocessing(df_original)

df_train, df_test = preprocessing.train_test_split_time(id_column='CREATION_DATE',test_size=0.2)

In [12]:
len(df_test)/ len(df_train)

0.25000097639849556

In [13]:
ls1 = list(df_train.sort_values(by='VISIT_ID')['VISIT_ID'])
ls2 = list(df_test.sort_values(by='VISIT_ID')['VISIT_ID'])
intersection = list(set(ls1) & set(ls2))
intersection ## to find the repeated transaction id between them

[254721]

In [15]:
path_date = f'data/{dir_date_day}-2024'; train_prq = path_date + '/train.parquet';test_prq = path_date + '/test.parquet'

df_train.to_parquet(train_prq)
df_test.to_parquet(test_prq)

In [16]:
print(list(df_train.columns))

['REQUEST_DATE', 'VISIT_NO', 'VISIT_DATE', 'EMERGENCY_INDICATOR', 'PROVIDER_DEPARTMENT_CODE', 'PROVIDER_DEPARTMENT', 'DOCTOR_SPECIALTY_CODE', 'DOCTOR_CODE', 'PATIENT_NO', 'EPISODE_NO', 'PATIENT_ID', 'MEMBERSHIP_NO', 'PATIENT_ID_TYPE', 'PATIENT_NATIONAL_IQAMA_ID', 'PATIENT_DOB', 'PATIENT_AGE', 'UNIT_OF_AGE', 'PATIENT_NATIONALITY', 'PATIENT_MARITAL_STATUS', 'PATIENT_GENDER', 'PATIENT_MOBILE_NO', 'PATIENT_BADGE_NO', 'HIS_INSURANCE_CODE', 'POLICY_HOLDER_NO', 'CLAIM_TYPE', 'BATCH_NO', 'TOTAL_CLAIMED_AMOUNT', 'TOTAL_CLAIMED_AMOUNT_SAR', 'TOTAL_NET_AMOUNT', 'TOTAL_DISCOUNT', 'TOTAL_DEDUCTIBLE', 'TOTAL_PATIENT_VATAMOUNT', 'TOTAL_NET_VAT_AMOUNT', 'CLAIM_NO', 'STAT_END_DATE', 'AMEND_BY_USER', 'AMEND_LAST_DATE', 'PAYER_LICENSE', 'DOCTOR_LICENSE', 'DEPARTMENT_TYPE', 'TREATMENT_TYPE', 'CLAIM_INVOICE_NO', 'STAT_INVOICE_NO', 'PURCHASER_CODE', 'STATUS', 'API_TRANS_ID', 'SUBMIT_CLAIM_OUTCOME', 'SUBMIT_CLAIM_MESSAGE', 'RES_STATUS', 'RES_CLAIM_ID', 'CONSULTATION_INVOICE_NO', 'TREATMENT_SUB_TYPE', 'ELIGIB

In [17]:
df_original.to_csv(f'data/{dir_date_day}-2024/df.csv',index=False)